In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pvlib
from pvlib import clearsky, atmosphere, solarposition
from pvlib.iotools import read_tmy3
from pvlib.pvsystem import PVSystem, FixedMount, Array
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.location import Location
import datetime
import pytz
import os
import itertools
import inspect
import calendar
import h5py
from pvlib import pvsystem, location, modelchain, iotools,tools
import pathlib
from dataclasses import dataclass
from pvlib import tracking
from netCDF4 import num2date
from requests.exceptions import HTTPError
from xml.etree.ElementTree import ParseError
from pvlib.irradiance import campbell_norman, get_extra_radiation, disc, louche, erbs_driesse, ghi_from_poa_driesse_2023
from pvlib.irradiance import _liujordan, get_total_irradiance
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS
import warnings
from pvlib._deprecation import deprecated
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
from scipy.optimize import least_squares

In [2]:
#BSRN QC
data_path ='STEL_2015_2016_2018_2019C_DNI.csv'
df = pd.read_csv(data_path, index_col=[0], parse_dates=[0])
#df.index = df.index.tz_localize('Africa/Johannesburg')  # Make the index timezone aware
original_entries = df.shape[0]
df.head(2)  # Print the first two lines of the DataFrame

,DNI,extraterrestrial_radiation,zenith_angle,airmass,cos_zenith
DATE,,,,,
2015-01-01 07:31:00+02:00,719.565,1414.91335,68.777384,2.746518,0.361993
2015-01-01 07:32:00+02:00,723.178,1414.91335,68.576732,2.722327,0.365255


In [3]:
# Define the Schulze model function
def schulze_model(airmass, a, b):
    return a * (b ** airmass)

# Generate some example data for df['airmass']
airmass = df['airmass']
dni_observed = df['DNI']

# Check for NaNs
if df['airmass'].isnull().values.any() or df['DNI'].isnull().values.any():
    print("Data contains NaNs")
    
# Check for infs
elif np.isinf(df['airmass']).values.any() or np.isinf(df['DNI']).values.any():
    print("Data contains infs")

# Remove NaNs and infs
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# Fit the Schulze model to the data to calibrate parameters a and b
popt, pcov = curve_fit(schulze_model, df['airmass'], df['DNI'], p0=[1127, 0.888])

# Extract the calibrated parameters
a_calibrated, b_calibrated = popt

print(f"Calibrated parameters: a = {a_calibrated}, b = {b_calibrated}")

Data contains NaNs
Calibrated parameters: a = 1126.6649752311632, b = 0.8660062158838217
